In [81]:
import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import pandas as pd

from utils import predicted_test_data_to_result_csv
from keras import layers, losses, Input, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Activation, Flatten
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import Adam, SGD

In [82]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [83]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [84]:
train_path = "data/base/goodreads_train.csv"
result_path = "data/base/goodreads_test.csv"

In [85]:
df = pd.read_csv(train_path, sep=",")

Drop all lines with 0 in rating column

In [86]:
index = df[(df['rating'] == 0)].index
df.drop(index, inplace=True)
df.reset_index(inplace=True, drop=True)

Targets DataFrames

In [87]:
targets = df.pop('rating')
targets = targets - 1
# targets = tf.keras.utils.to_categorical(targets)

Features DataFrames

In [88]:
features_names = ['review_text']
features = df[features_names]
tf.convert_to_tensor(features)

<tf.Tensor: shape=(869012, 1), dtype=string, numpy=
array([[b'This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. \n It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I\'ve read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc. \n It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. (view spoiler)[Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though. \n But what would happen if our SETI stations received a message - if we found someone was out

In [89]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_spoilers = tf.strings.regex_replace(lowercase, '\*\* spoiler alert \*\*', ' ')
    return tf.strings.regex_replace(stripped_spoilers,
                                    '[%s]' % re.escape(string.punctuation),
                                    '')

In [90]:
max_features = 5000  # Maximum vocab size.
sequence_length = 100

In [91]:
vectorized_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [92]:
vectorized_layer.adapt(features)

In [93]:
epochs = 50
model_nb = 2

embedding_dim = 50
learning_rate = 0.0001
batch_size = 8000
dropout_rate = 0.0

In [94]:
input_text = Input(shape=(1,), dtype=tf.string)

vectorized_text = vectorized_layer(input_text)

embedding_layer = Embedding(max_features + 1, embedding_dim, input_length=sequence_length)(vectorized_text)

x_shortcut = embedding_layer

#### Main path ####
# First
x = Conv1D(64, 3, activation='relu', padding = 'valid')(embedding_layer)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Second
x = Conv1D(128, 5, activation='relu', padding = 'valid')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Third
x = Conv1D(256, 5, activation='relu', padding = 'valid')(x)
x = BatchNormalization()(x)

#### Shortcut path ####
x_shortcut = Conv1D(256, 11, activation='relu', padding = 'valid')(x_shortcut)
x_shortcut = BatchNormalization()(x_shortcut)

# x and x_shortcut addition
x = x + x_shortcut

global_max_pooling = GlobalMaxPooling1D()(x)

relu = Dense(32, activation='relu')(global_max_pooling)

output = Dense(5, activation='softmax')(relu)

resnet_model = Model(input_text, output)

resnet_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization_7 (TextVect  (None, 100)         0           ['input_8[0][0]']                
 orization)                                                                                       
                                                                                                  
 embedding_4 (Embedding)        (None, 100, 50)      250050      ['text_vectorization_7[0][0]']   
                                                                                                  
 conv1d_16 (Conv1D)             (None, 98, 64)       9664        ['embedding_4[0][0]']      

In [95]:
resnet_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate=learning_rate),
                  metrics=sparse_categorical_accuracy)

In [96]:
exp_name = f'resnet_model_{model_nb}_lr_{learning_rate}_bs_{batch_size}_dr_{dropout_rate}'

In [ ]:
resnet_model.fit(features,
              targets,
              validation_split=0.25,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[tf.keras.callbacks.TensorBoard("logs/resnets/" + exp_name)])

Epoch 1/50
82/82 [==============================] - 70s 804ms/step - loss: 2.1018 - sparse_categorical_accuracy: 0.2950 - val_loss: 1.4902 - val_sparse_categorical_accuracy: 0.3140
Epoch 2/50
82/82 [==============================] - 64s 780ms/step - loss: 1.4809 - sparse_categorical_accuracy: 0.3578 - val_loss: 1.4810 - val_sparse_categorical_accuracy: 0.3140
Epoch 3/50
82/82 [==============================] - 64s 780ms/step - loss: 1.3872 - sparse_categorical_accuracy: 0.3928 - val_loss: 1.4809 - val_sparse_categorical_accuracy: 0.3140
Epoch 4/50
82/82 [==============================] - 64s 779ms/step - loss: 1.3229 - sparse_categorical_accuracy: 0.4215 - val_loss: 1.4710 - val_sparse_categorical_accuracy: 0.3140
Epoch 5/50
82/82 [==============================] - 64s 777ms/step - loss: 1.2727 - sparse_categorical_accuracy: 0.4441 - val_loss: 1.4405 - val_sparse_categorical_accuracy: 0.3141
Epoch 6/50
82/82 [==============================] - 64s 777ms/step - loss: 1.2303 - sparse_cate

In [ ]:
df_test = pd.read_csv(result_path, sep=",")

df_test_modified = df_test.drop(columns=[
    'user_id',
    'book_id',
    'review_id',
    'date_added',
    'date_updated',
    'read_at',
    'started_at',
    'n_votes',
    'n_comments'
], inplace=False)

In [ ]:
predicted_test_data = resnet_model.predict(df_test_modified)

In [ ]:
df_test.head()

In [ ]:
predicted_test_data_to_result_csv(df_test, predicted_test_data, exp_name)